# U.S. Energy and Energy Infrastructure Map

This notebook documents the step-by-step process of developing an interactive GIS-based mapping system for U.S. energy infrastructure. The system provides a graphical user interface (GUI) that enables users to evaluate geospatial and environmental factors that influence power plant design, grid layout, and supporting infrastructure.

In addition to its value for conceptual design and site selection, this resource supports logistics planning for the construction and transportation of large-scale energy facilities.

The current prototype is implemented using the `ipyleaflet` package within a Jupyter environment, with plans to evolve the system into a standalone web application using `folium` or other web mapping tools. This notebook also tracks the provenance of all geospatial data and includes preprocessing (wrangling) steps required to transform raw datasets into a usable format. The final outputs include structured local files intended for reuse in the future web application.

## 1. Key Features Under Development

This tool aims to integrate and visualize geospatial datasets relevant to siting, design, and analysis of power infrastructure, including:

- Road networks
- Topographic features and elevation data
- Seismic hazard zones
- Prevailing wind and weather patterns
- Time-dependent temperature profiles
- Population density distributions
- Major transportation infrastructure (e.g., highways, railways)
- Power grid layout and metadata
- Existing power plant locations and attributes
- Wind corridors and wind energy potential
- Solar irradiance maps


## 2. Import Packages
The following python packages need to be imported to enable this notebook

In [1]:
from ipyleaflet import Map, TileLayer, basemap_to_tiles, basemaps, WidgetControl, GeoJSON
import geopandas as gpd
import pandas as pd
import json
import os
from ipywidgets import Dropdown, Layout
from IPython.display import display
from typing import Dict
from traitlets import TraitError
import re
from datetime import datetime

## 3. Data Selection and Wrangling
This section documents the sources of geospatial data, how the data is obtained, and—most importantly—how it is cleaned and transformed into a usable format for the application. Where available, shapefiles are used to preserve geospatial attributes essential for interactive mapping. However, shapefiles often contain a wide range of metadata, much of which may not be relevant to this project. The following subsections describe the types of data selected, how unnecessary information is filtered out, and how the remaining data is wrangled into a streamlined, application-ready format.

### 3.1. State Boundaries

To display U.S. state boundaries on the map—and to support future geospatial analysis—we begin by downloading official boundary data from the [U.S. Census Bureau](https://www.census.gov/cgi-bin/geo/shapefiles/index.php). This data is available under the **“States”** dropdown menu on the linked page. It is provided as a collection of shapefiles, with the main file titled `tl_2024_us_state.shp`.

Once downloaded, the shapefiles are placed in the `Data/States` directory within this project. The following code demonstrates how the data is read into memory, filtered, and processed to produce a simplified GeoJSON file. This preprocessing step ensures the transformation only needs to be performed once.

As part of the wrangling process:

- The dataset is reduced to essential columns: state identifier (`GEOID`), geospatial identifier (`STATEFP`), postal abbreviation (`STUSPS`), and geometry.
- The area of each state is computed in square kilometers, using a projection suitable for area calculation.
- The centroid of each state is calculated to serve as a focal point for snapping or centering the map on that state in future functionality.

The final output is saved as `stateBoundaries.json` in the `jsonFiles` directory.

The code block below performs this complete process:

In [2]:
# Paths
shapefile_path = "RawData/States/tl_2024_us_state.shp"
geojson_dir = "FinalData"
geojson_stateGeometry = os.path.join(geojson_dir, "stateGeometry.json")

# ------------------------------------------------------------------------------
# 1. Ensure jsonFiles directory exists
os.makedirs(geojson_dir, exist_ok=True)

# ------------------------------------------------------------------------------
# 2. Read only required columns: GEOID, STUSPS, geometry
# Note: Use `usecols` if supported, else select after loading
gdf_states = gpd.read_file(shapefile_path)[["STATEFP", "GEOID", "STUSPS", "geometry"]]

# ------------------------------------------------------------------------------
# 3. Rename STUSPS to State
gdf_states = gdf_states.rename(columns={"STUSPS": "State"})

# ------------------------------------------------------------------------------
# 4. Compute area and centroid
# Set CRS if missing
gdf_states.set_crs("EPSG:4269", inplace=True)

# Reproject to Albers Equal Area for accurate area and centroid calculations
gdf_states = gdf_states.to_crs("EPSG:5070")
gdf_states["area_km2"] = gdf_states.geometry.area / 1e6
gdf_states["centroid"] = gdf_states.geometry.centroid

# Reproject back to WGS84 for display (geometry and centroid)
gdf_states = gdf_states.to_crs("EPSG:4326")
gdf_states["centroid"] = gdf_states["centroid"].to_crs("EPSG:4326")

# ------------------------------------------------------------------------------
# 5. Delete existing GeoJSON if it exists
if os.path.exists(geojson_stateGeometry):
    os.remove(geojson_stateGeometry)
    print(f"Removed old file: {geojson_stateGeometry}\n")

# ------------------------------------------------------------------------------
# 6. Convert centroid to WKT string
gdf_states["centroid_wkt"] = gdf_states["centroid"].to_wkt()

# ------------------------------------------------------------------------------
# 7. Drop the second geometry column to avoid conflict
gdf_states = gdf_states.drop(columns=["centroid"])

# ------------------------------------------------------------------------------
# 8. Display the first few rows
print("Final GeoPandas DataFrame")
print("-------------------------")
display(gdf_states.head())

# ------------------------------------------------------------------------------
# 9. Write to GeoJSON
gdf_states.to_file(geojson_stateGeometry, driver="GeoJSON")
print(f"GeoJSON saved to {geojson_stateGeometry}")

Removed old file: FinalData/stateGeometry.json

Final GeoPandas DataFrame
-------------------------


,STATEFP,GEOID,State,geometry,area_km2,centroid_wkt
0,54,54,WV,"POLYGON ((-77.75438 39.33346, -77.75422 39.333...",62755.438109,POINT (-80.623488 38.640815)
1,12,12,FL,"MULTIPOLYGON (((-83.10874 24.62949, -83.10711 ...",184934.309000,POINT (-82.495966 28.413869)
2,17,17,IL,"POLYGON ((-87.89243 38.28285, -87.89334 38.282...",149995.054520,POINT (-89.149853 40.09991)
3,27,27,MN,"POLYGON ((-95.31991 48.99892, -95.31778 48.998...",225182.049911,POINT (-94.198929 46.310806)
4,24,24,MD,"POLYGON ((-75.756 39.24607, -75.75578 39.24335...",32131.069585,POINT (-76.679027 38.946076)


GeoJSON saved to FinalData/stateGeometry.json


### 3.2. County and Population Data

Geographic data describing the location and geometry of each U.S. county can be obtained from the [U.S. Census Bureau](https://www.census.gov/cgi-bin/geo/shapefiles/index.php).  
From the dropdown menu, select **"Counties (and equivalent)"**. This will display a download button for the official shapefiles covering all U.S. counties.

For this project, the shapefile set was saved to the `Data/Population` directory under the filename `tl_2024_us_county.shp`.  

The county shapefile is read into a GeoPandas DataFrame, merged with state name information from the state boundaries dataset, and then processed to include two additional attributes:  

- **Area (`area_km2`)** — Calculated in square kilometers using an equal-area projection.  
  This value will be used in future **population density calculations**.  
- **Centroid (`centroid_lon`, `centroid_lat`)** — The geographic center of each county, reprojected into WGS84 coordinates for compatibility with web mapping tools.  
  These values will be used to **snap the map to specific counties** in future functionality.

By storing only the area and centroid coordinates alongside the county geometry, we preserve efficiency and avoid unnecessary extra geometry columns that can complicate GeoJSON export.  The code shown below demonstrates the process of wrangling the county data into the required format and provides a print oput of the first few lines of the data frame.

In [3]:
# 1) Set path to read file in
countyShapefilePath = "RawData/Population/tl_2024_us_county.shp" # "Select from Counties and Subdivisions"

# 2) Read the shape file into a GeoPandas data frame
county_gdf = gpd.read_file(countyShapefilePath)[["STATEFP", "GEOID", "COUNTYFP", "NAME", "geometry"]]

# 3) Merge county_gdf with gdf_states to add state names
county_gdf = county_gdf.merge(
    gdf_states[["STATEFP", "State"]],  # Only the columns you need
    on="STATEFP",
    how="left"  # Keep all counties, even if no match (shouldn't happen with Census data)
)

# 4) Ensure source CRS is set (TIGER/Line is NAD83)
county_gdf = county_gdf.set_crs("EPSG:4269", allow_override=True)

# 5) Work in an equal-area projection for area/centroid calculations
county_proj = county_gdf.to_crs("EPSG:5070")  # USA_Contiguous_Albers_Equal_Area

# 6) Compute area (km²) and centroids in projected CRS
county_gdf["area_km2"] = county_proj.geometry.area / 1e6

centroids_proj = county_proj.geometry.centroid  # centroid in EPSG:5070
centroids_wgs84 = gpd.GeoSeries(centroids_proj, crs="EPSG:5070").to_crs("EPSG:4326")

# 7) Add centroid columns as lon/lat (avoids second geometry column conflict)
county_gdf["centroid_lon"] = centroids_wgs84.x
county_gdf["centroid_lat"] = centroids_wgs84.y

# 8) Keep display/mapping geometry in WGS84 for ipyleaflet/Leaflet
county_gdf = county_gdf.to_crs("EPSG:4326")

# 9) Display the first few rows
display(county_gdf.head())

,STATEFP,GEOID,COUNTYFP,NAME,geometry,State,area_km2,centroid_lon,centroid_lat
0,31,31039,039,Cuming,"POLYGON ((-96.55525 41.82892, -96.55524 41.827...",NE,1488.335592,-96.787400,41.916312
1,53,53069,069,Wahkiakum,"POLYGON ((-123.72755 46.2645, -123.72756 46.26...",WA,742.545319,-123.433359,46.291139
2,35,35011,011,De Baca,"POLYGON ((-104.89337 34.08894, -104.89337 34.0...",NM,6045.909493,-104.412089,34.342297
3,31,31109,109,Lancaster,"POLYGON ((-96.68493 40.5233, -96.69219 40.5231...",NE,2192.120120,-96.687757,40.783894
4,31,31129,129,Nuckolls,"POLYGON ((-98.2737 40.1184, -98.27374 40.1224,...",NE,1491.363731,-98.047185,40.176298


Next, we download a list of the population for all U.S. counties from the [U.S. Census Bureau](https://www.census.gov/data/tables/time-series/demo/popest/2020s-counties-total.html).  In this case, the file `co-est2024-pop.xlsx` was saved to the `Data/Population` directory.

Unfortunately, the Excel file has a **non-standard format** with multiple level headers.  In order to make the process of reading the file in a bit easier, it was manually reformatted to ensure it had a Geographic Area column and a 2024 column containing the population for each Geographic area.

After reading this file into a Pandas DataFrame, the following wrangling steps are performed:

1. **Locate the actual header row** by searching for the `"Geographic Area"` label.  
2. **Filter out national and state-level rows**, keeping only county-level entries.  
3. **Split the `"Geographic Area"` column** into `County Name` and `State Name`.  
4. **Map state names to their two-letter abbreviations** for joining with geospatial data.  
5. **Normalize county names** by removing suffixes like “County”, “Parish”, or “Borough” so they match the shapefile format.  

This process ensures that the population data aligns perfectly with the geometry information in the county shapefile, enabling calculations such as population density and spatial queries.  The code to read in and process the relevant data is shown below with a brief display for the format of the output table.


In [5]:
csv_path = "RawData/Population/co-est2024-pop.csv"

# State name → abbreviation
STATE_ABBR = {
    "Alabama":"AL","Alaska":"AK","Arizona":"AZ","Arkansas":"AR","California":"CA","Colorado":"CO",
    "Connecticut":"CT","Delaware":"DE","District of Columbia":"DC","Florida":"FL","Georgia":"GA",
    "Hawaii":"HI","Idaho":"ID","Illinois":"IL","Indiana":"IN","Iowa":"IA","Kansas":"KS","Kentucky":"KY",
    "Louisiana":"LA","Maine":"ME","Maryland":"MD","Massachusetts":"MA","Michigan":"MI","Minnesota":"MN",
    "Mississippi":"MS","Missouri":"MO","Montana":"MT","Nebraska":"NE","Nevada":"NV","New Hampshire":"NH",
    "New Jersey":"NJ","New Mexico":"NM","New York":"NY","North Carolina":"NC","North Dakota":"ND",
    "Ohio":"OH","Oklahoma":"OK","Oregon":"OR","Pennsylvania":"PA","Rhode Island":"RI","South Carolina":"SC",
    "South Dakota":"SD","Tennessee":"TN","Texas":"TX","Utah":"UT","Vermont":"VT","Virginia":"VA",
    "Washington":"WA","West Virginia":"WV","Wisconsin":"WI","Wyoming":"WY"
}

# Suffixes to strip from county-equivalents
SUFFIXES = [
    " City and Borough", " Municipality of Anchorage", " County", " Parish", " Borough", " Census Area", " Municipality",
    " City", " city", " Planning Region"
]

# Counties that legitimately end in "City" and should NOT be altered
CITY_EXCEPTIONS = {
    "Charles City", "James City", "Carson City", "Falls Church City", "Fairfax City",
    "Hampton City", "Newport News City", "Norfolk City", "Richmond City", "Roanoke City",
    "Suffolk City", "Virginia Beach City"  # Add any others as needed
}

def normalize_county_name(name: str) -> str:
    """Strip known suffixes from county names unless they are legitimate names."""
    if pd.isna(name):
        return name
    name = str(name).strip()

    # Only strip suffix if not in the exceptions list
    for suf in SUFFIXES:
        if name.endswith(suf) and name not in CITY_EXCEPTIONS:
            name = name[:-len(suf)]
            break

    return re.sub(r"\s+", " ", name).strip()

# --- 1) Read CSV ---
# ---------- Build pop_df from CSV (Geographic Area + 2024) ----------
pop = pd.read_csv(
    csv_path,
    usecols=["Geographic Area", "2024"],
    dtype={"Geographic Area": "string"},
    thousands=","  # handle comma-formatted numbers
)

# --- 2) Basic cleanup ---
pop.columns = [c.strip() for c in pop.columns]  # strip header whitespace

# Ensure needed columns exist
if "Geographic Area" not in pop.columns or "2024" not in pop.columns:
    raise RuntimeError(f"Expected 'Geographic Area' and '2024' in columns. Found: {list(pop.columns)}")

# --- 3) Keep only the columns we want ---
pop = pop[["Geographic Area", "2024"]].copy()

# --- 4) Strip leading '.' and whitespace ---
pop["Geographic Area"] = pop["Geographic Area"].str.lstrip(".").str.strip()

# --- 5) Split into County / State ---
split = pop["Geographic Area"].str.rsplit(",", n=1, expand=True)
pop["County"] = split[0].apply(normalize_county_name)
pop["state_full"] = split[1].str.strip()

# --- 6) Map state to abbreviation ---
pop["State"] = pop["state_full"].map(STATE_ABBR)

# --- 7) Rename population column and ensure numeric ---
pop = pop.rename(columns={"2024": "Population"})
pop["Population"] = pd.to_numeric(pop["Population"], errors="coerce")

# --- 8) Keep only valid entries ---
pop_df = pop.loc[pop["State"].notna() & pop["County"].notna(), ["County", "State", "Population"]].reset_index(drop=True)

# --- 9) Preview ---
display(pop_df.head())
print(f"Rows kept: {len(pop_df):,}")

,County,State,Population
0,Autauga,AL,61464
1,Baldwin,AL,261608
2,Barbour,AL,24358
3,Bibb,AL,22258
4,Blount,AL,60163


Rows kept: 3,144


Finally we merge the `pop` data frame with the `county_gdf` dataframe to get a final dataframe containing GIS data for each county along with its population and population density.  The final database is shown below.

In [8]:
# ---------- Prepare county_gdf for a clean join ----------
county_gdf["NAME_norm"] = county_gdf["NAME"].astype(str).str.strip().apply(normalize_county_name)
county_gdf["State"]     = county_gdf["State"].astype(str).str.strip()

# Make a safe copy of pop_df with unique column names
pop_join = (
    pop_df.rename(columns={"County": "NAME_norm", "Population": "Population_2024"})
          .assign(State=lambda d: d["State"].astype(str).str.strip())
          [["NAME_norm", "State", "Population_2024"]]
)

# Drop any old population-related columns to avoid suffixes
for col in ["Population", "Population_x", "Population_y", "Population_2024", "pop_density_km2"]:
    if col in county_gdf.columns:
        county_gdf = county_gdf.drop(columns=col)

# ---------- Merge population onto county_gdf ----------
county_gdf = county_gdf.merge(
    pop_join,
    how="left",
    on=["NAME_norm", "State"]
)

# ---------- Compute population density ----------
county_gdf["pop_density_km2"] = county_gdf["Population_2024"] / county_gdf["area_km2"]

# ---------- Drop temporary columns ----------
county_gdf = county_gdf.drop(columns=["NAME_norm"], errors="ignore")

# ---------- Merge Metrics --------------------
EXCLUDE_STATES = {"AS", "MP", "GU", "PR", "VI"}

unmatched = (
    county_gdf[
        county_gdf["Population_2024"].isna()
        & ~county_gdf["State"].isin(EXCLUDE_STATES)
    ][["STATEFP", "GEOID", "NAME", "State"]]
    .sort_values(["State", "NAME"])
)

print(f"\nUnmatched counties (no population found, excluding AS/MP/GU/PR/VI): {len(unmatched)}")
pd.set_option("display.max_rows", None)
if len(unmatched) > 0:
    display(unmatched)
    pd.reset_option("display.max_rows")

# (Optional) See how many were excluded because they're in territories
excluded_territories = (
    county_gdf[
        county_gdf["Population_2024"].isna()
        & county_gdf["State"].isin(EXCLUDE_STATES)
    ][["STATEFP", "GEOID", "NAME", "State"]]
    .sort_values(["State", "NAME"])
)
print(f"Unmatched in territories (AS/MP/GU/PR/VI), not counted above: {len(excluded_territories)}")

# Preview a few merged rows
display(county_gdf.head())

out_dir = "FinalData"
out_path = os.path.join(out_dir, "county_population.gpkg")
layer_name = "county_population"

# 1) Ensure output directory exists
os.makedirs(out_dir, exist_ok=True)

# 2) (Optional) clean up column names & ensure CRS
county_gdf.columns = [str(c) for c in county_gdf.columns]  # avoid non-string names
if county_gdf.crs is None:
    # set to your display CRS if missing; you used WGS84 earlier
    county_gdf = county_gdf.set_crs("EPSG:4326", allow_override=True)

# 3) If file exists, remove it so we start fresh
if os.path.exists(out_path):
    os.remove(out_path)
    print(f"Deleted existing file: {out_path}")

# 4) Write to GeoPackage
try:
    county_gdf.to_file(out_path, layer=layer_name, driver="GPKG")
    print(f"Data saved to '{out_path}' (layer='{layer_name}')")
except Exception as e:
    print("Failed to write GeoPackage.")
    print(f"cwd: {os.getcwd()}")
    print(f"Have write permission to '{out_dir}': {os.access(out_dir, os.W_OK)}")
    raise

# Remove existing file to avoid overwrite conflicts
if os.path.exists(out_path):
    os.remove(out_path)
    print(f"Deleted existing file: {out_path}")

# Save GeoDataFrame to GeoPackage
county_gdf.to_file(out_path, driver="GPKG")
print(f"Data saved to '{out_path}'")


Unmatched counties (no population found, excluding AS/MP/GU/PR/VI): 0
Unmatched in territories (AS/MP/GU/PR/VI), not counted above: 91


,STATEFP,GEOID,COUNTYFP,NAME,geometry,State,area_km2,centroid_lon,centroid_lat,Population_2024,pop_density_km2
0,31,31039,039,Cuming,"POLYGON ((-96.55525 41.82892, -96.55524 41.827...",NE,1488.335592,-96.787400,41.916312,8952.0,6.014773
1,53,53069,069,Wahkiakum,"POLYGON ((-123.72755 46.2645, -123.72756 46.26...",WA,742.545319,-123.433359,46.291139,4800.0,6.464252
2,35,35011,011,De Baca,"POLYGON ((-104.89337 34.08894, -104.89337 34.0...",NM,6045.909493,-104.412089,34.342297,1657.0,0.274070
3,31,31109,109,Lancaster,"POLYGON ((-96.68493 40.5233, -96.69219 40.5231...",NE,2192.120120,-96.687757,40.783894,332857.0,151.842500
4,31,31129,129,Nuckolls,"POLYGON ((-98.2737 40.1184, -98.27374 40.1224,...",NE,1491.363731,-98.047185,40.176298,4094.0,2.745139


Deleted existing file: FinalData/county_population.gpkg
Data saved to 'FinalData/county_population.gpkg' (layer='county_population')
Deleted existing file: FinalData/county_population.gpkg
Data saved to 'FinalData/county_population.gpkg'


## 4. Create Base Map
This section creates the range of basemap options that the user can select from.  The default option is `Esri Satellite`; however, the user can select from `OpenStreetMap` or `OpenTopoMap`.  All three of these options are inherent in the `ipyleaflet` Python package.

In [40]:
# Define Functions

def on_basemap_change(
    change: dict,
    m: Map,
    dropdown: Dropdown,
    basemap_options: Dict[str, Dict]
) -> None:
    """Replace the current base layer with a new one selected from the dropdown."""
    if change['name'] == 'value':
        try:
            new_basemap: TileLayer = basemap_to_tiles(basemap_options[change['new']])
            new_basemap.name = change['new']
            m.substitute_layer(m.layers[1], new_basemap)
        except TraitError as e:
            print(f"Failed to switch basemap: {e}")
            
# ================================================================================
# ================================================================================
# Main code

# Available basemap options
basemap_options: Dict[str, Dict] = {
    "Esri Satellite": basemaps.Esri.WorldImagery,
    "OpenStreetMap": basemaps.OpenStreetMap.Mapnik,
    "OpenTopoMap": basemaps.OpenTopoMap
}

# Create initial base map layer
initial_basemap: TileLayer = basemap_to_tiles(basemap_options["Esri Satellite"])

# Create the map object
m: Map = Map(
    center=(39.8283, -98.5795),
    zoom=5,
    scroll_wheel_zoom=True,
    layout=Layout(height='700px')
)
m.add_layer(initial_basemap)

# Create the dropdown widget
basemap_dropdown: Dropdown = Dropdown(
    options=list(basemap_options.keys()),
    value="Esri Satellite",
    layout=Layout(width='200px')
)

# Register observer with bound arguments using a lambda
basemap_dropdown.observe(
    lambda change: on_basemap_change(change, m, basemap_dropdown, basemap_options),
    names='value'
)

# Add dropdown widget to map
dropdown_control: WidgetControl = WidgetControl(widget=basemap_dropdown, position='topright')
m.add_control(dropdown_control)


# Load the local GeoJSON file
with open(geojson_stateGeometry, "r") as f:
    states_geojson_data = json.load(f)

# Create the GeoJSON layer for state boundaries
state_boundaries = GeoJSON(
    data=states_geojson_data,
    name="State Boundaries",
    style={
        'color': 'black',       # Line color
        'weight': 0.6,          # Line thickness
        'fillOpacity': 0.0      # Transparent fill
    },
)

# Add it to your map
m.add_layer(state_boundaries)

# Display the final map
display(m)


Map(center=[39.8283, -98.5795], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…